In [1]:
# https://huggingface.co/docs/peft/index

import os 

### Utils
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def split_strings(strings):
    first_parts = []
    second_parts = []
    for string in strings:
        split_index = len(string) // 2
        while split_index < len(string) and not string[split_index].isspace():
            split_index += 1
        first_part = string[:split_index].rstrip()
        second_part = string[split_index:].lstrip()
        first_parts.append(first_part)
        second_parts.append(second_part)
    return first_parts, second_parts
    
### 

math_example_path = os.getcwd() + '/data/What_Is_Mathematics_An_Elementary_Approach_to_Ideas_and_Methods.txt'

with open(math_example_path, "r",  encoding="utf8") as f:
     math_example_text = f.read()

math_example_text = math_example_text.replace("\n", "")
math_sentences = math_example_text.split(".")

In [2]:
print(len(math_sentences))
math_sentences = [item for item in math_sentences if 80 < len(item) < 100]
X, Y = split_strings(math_sentences)
print(math_sentences[1])

9421
 But it is not a concession to the dangerous tendency toward dodging all exertion


In [3]:
tst_ind = 30
print(math_sentences[tst_ind], "\n X: ", X[tst_ind], "\n Y: ", Y[tst_ind])

 For a « 20, we need twenty words for the digits, plus words for 20 and 400, making a total of 22 
 X:   For a « 20, we need twenty words for the digits, 
 Y:  plus words for 20 and 400, making a total of 22


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = torch.device('cuda')

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium", padding_side='left')
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
tokenizer.pad_token = tokenizer.eos_token

model.to(device)

# for step in range(5):
#     input_str = input(">> User:") + tokenizer.eos_token
#     new_user_input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)

#     bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1).to(device) if step > 0 else new_user_input_ids

#     chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id).to(device)

#     print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

C:\Users\tempdelta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\tempdelta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [5]:
def vis_res(str_Xs, vis_i = 0):
    encoded_test_X = tokenizer.batch_encode_plus(str_Xs, 
                                         add_special_tokens=False, 
                                         return_tensors='pt', 
                                         padding=True) # X is encoded Y isnt !
    
    ###
    test_item = encoded_test_X["input_ids"][vis_i]
    print(test_item)
    print(tokenizer.decode(test_item))
    
    output = model.generate(encoded_test_X["input_ids"].to(device),  
                                max_new_tokens=200,   
                                pad_token_id=tokenizer.eos_token_id)
    
    print(output[vis_i], "\n ")
    print(tokenizer.decode(output[vis_i], skip_special_tokens=True))

#vis_res(["Do you love me?" + tokenizer.eos_token])
vis_res([" But it is not a concession to the dangerous<|endoftext|>" + tokenizer.eos_token])

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


tensor([  887,   340,   318,   407,   257, 33130,   284,   262,  4923, 50256,
        50256])
 But it is not a concession to the dangerous<|endoftext|><|endoftext|>
tensor([  887,   340,   318,   407,   257, 33130,   284,   262,  4923, 50256,
        50256,    40,  1101,   407,  1654,   644,   345,   821,  2111,   284,
          910,   764, 50256], device='cuda:0') 
 
 But it is not a concession to the dangerousI'm not sure what you're trying to say.


In [6]:
# should be called only once
is_called = False

if is_called is False:
    X = [s + tokenizer.eos_token for s in X]
    Y = [s + tokenizer.eos_token for s in Y]
    is_called = True

In [41]:
tt_x = tokenizer(X[0:33], padding=True, truncation=True, return_tensors="pt")

input_ids = tt_x["input_ids"] 

t_inp = torch.Tensor(input_ids)

print("str inp", X[0])

print("torch t:", t_inp[0])

str inp  Unfortunately, professional repre¬ sentatives<|endoftext|>
torch t: tensor([50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256,  8989,    11,  4708,  1128,   260,   126,   105,
         1908,  2929, 50256])


In [15]:
vis_i = 1
encoded_test_X = tokenizer.batch_encode_plus(X[0:99], 
                                     add_special_tokens=False, 
                                     return_tensors='pt', 
                                     padding=True) # X is encoded Y isnt !

###
test_item = encoded_test_X["input_ids"][vis_i]
print(test_item)
print(tokenizer.decode(test_item))

output = model.generate(encoded_test_X["input_ids"].to(device),  
                            max_new_tokens=200,   
                            pad_token_id=tokenizer.eos_token_id)

print(output[vis_i], "\n ")
print(tokenizer.decode(output[vis_i], skip_special_tokens=True))


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


tensor([50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,   887,
          340,   318,   407,   257, 33130,   284,   262,  4923, 50256])
<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|> But it is not a concession to the dangerous<|endoftext|>
tensor([50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,   887,
          340,   318,   407,   257, 33130,   284,   262,  4923, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256], device='cuda:0') 
 
 But it is not a concession to the dangerous


In [16]:
encoded_test_X["input_ids"].shape

torch.Size([99, 29])

In [ ]:
import numpy as np
#np.random.seed(309)

# Set padding to the left side as the model is a decoder-only architecture

encoded_X = tokenizer.batch_encode_plus(X, add_special_tokens=False, return_tensors='pt', padding=True)
encoded_Y = tokenizer.batch_encode_plus(Y, add_special_tokens=False, return_tensors='pt', padding=True)

BATH_SIZE = 10

x_batches = list(chunks(encoded_X["input_ids"], BATH_SIZE))
y_batches = list(chunks(encoded_Y["input_ids"], BATH_SIZE))

In [ ]:
test_index = 0
test_item = encoded_X["input_ids"][test_index]

print(X[test_index])
print(test_item)
print(tokenizer.decode(test_item))

In [ ]:
output = model.generate(x_batches[0].to(device),  max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)

In [ ]:
print(output[test_index], "\n ")
print(tokenizer.decode(output[test_index]))

In [ ]:
# Prepare input data

tst_i = 3
test_str = test_X[tst_i]
input_ids = tokenizer.encode(test_str,
                             add_special_tokens=False, 
                             return_tensors='pt', 
                             padding=True).to(device)

# Generate text using generation parameters
output = llm_model.generate(input_ids , 
                            max_new_tokens=100, 
                            pad_token_id=tokenizer.eos_token_id)

# Decode and print the output
decoded_output = tokenizer.decode(output[0])
print(f"input: {test_str} \n")
print(f"input_ids: {input_ids} \n")
print(f"output: {output} \n")

print(f"decoded_output:{decoded_output}\n", )
print(f"expected output:{test_Y[tst_i]} ")

In [ ]:
import torch

optimizer = torch.optim.AdamW(llm_model.parameters(), lr=1e-5)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [ ]:
import mlflow


EPOCHS=10

mlflow.end_run()
gradient_accumulation_steps = 10
grad_counter = 0

with mlflow.start_run(run_name='final') as run:

    for epoch in range(EPOCHS):
        for index, batch in enumerate(x_batches[0:5]): 
            outputs = llm_model(batch, labels=y_batches[index])
            loss = outputs.loss
            loss.backward()

            if grad_counter % gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

                print("batch loss ", loss.item())

                
            grad_counter += 1
                            

        # log metric every epoch
        mlflow.log_metric('loss', loss.item())
        mlflow.log_metric('lr', get_lr(optimizer))

        
        output = llm_model.generate(tst_data["input_ids"].to(device))
        test_output = ""
        for i in range(output.shape[0]):
            tmp_str = tokenizer.decode(output[i])
            test_output += "\n" + tmp_str
        mlflow.log_text(test_output, f"test_{epoch}.txt")
        #### 

        
        # Save the fine-tuned model every epoch 
        print("MODEL saved loss ", loss)
        llm_model.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)

In [ ]:
import numpy as np
tst_text = np.random.choice(math_sentences, 30)
tst_data = tokenizer.batch_encode_plus(tst_text, add_special_tokens=True, return_tensors='pt', padding=True).to(device)
output = llm_model.generate(tst_data["input_ids"])

for i, text in enumerate(tst_text):
    print("input:", text)
    print("output", tokenizer.decode(output[i]))

In [ ]:
from transformers import TextDataset

train_dataset = TextDataset(
      tokenizer=tokenizer,
      file_path=math_example_path,
      block_size=128)

In [ ]:
from transformers import (
    TrainingArguments, 
    Trainer, 
    default_data_collator, 
)
import mlflow 
mlflow.end_run()

training_args = TrainingArguments(output_dir='test_trainer', 
                                  #evaluation_strategy='epoch',
                                  per_device_train_batch_size=1,
                                  per_device_eval_batch_size=1,
                                  gradient_accumulation_steps=20, #
                                  num_train_epochs = 2,
                                  fp16=False,)

trainer = Trainer(
    model=llm_model,
    args=training_args,
    train_dataset=dataset,
    #eval_dataset=dataset,
    tokenizer=tokenizer,
    # Data collator will default to DataCollatorWithPadding, so we change it.
    data_collator=default_data_collator,
    compute_metrics=None,
    preprocess_logits_for_metrics=None,
)

trainer.train()

In [ ]:
dataset